In [2]:
print("Hello friends...")

Hello friends...


In [5]:
import random

choices = ["rock", "paper", "scissors"]

print("🎮 Welcome to Rock–Paper–Scissors!")
print("Type rock, paper, or scissors to play.")
print("Type exit to stop the game.\n")

while True:
    user = input("Your choice: ").lower()

    if user == "exit":
        print("👋 Thanks for playing! Bye!")
        break

    if user not in choices:
        print("❌ Invalid choice! Try again.")
        continue

    computer = random.choice(choices)
    print(f"Computer chose: {computer}")

    if user == computer:
        print("🤝 It's a tie!\n")

    elif (user == "rock" and computer == "scissors") or \
         (user == "paper" and computer == "rock") or \
         (user == "scissors" and computer == "paper"):
        print("🎉 You win!\n")
    else:
        print("😢 You lose!\n")


🎮 Welcome to Rock–Paper–Scissors!
Type rock, paper, or scissors to play.
Type exit to stop the game.

👋 Thanks for playing! Bye!


In [ ]:
"""
party_games.py

Contains:
1) ASCII Snake (uses curses)
2) Shooting Game (text target)
3) Treasure Hunt (text adventure)
4) Memory / IQ Challenge
5) Roast Battle
6) Multiplayer Text Battle (2-player local)
7) Horror Escape (text adventure)

Run: python party_games.py
"""

import random
import time
import sys

# Try to import curses for snake; if not available on Windows prompt user to install windows-curses.
try:
    import curses
except Exception:
    curses = None

# -------------------------
# Utility functions
# -------------------------
def clear():
    if sys.platform.startswith("win"):
        _ = __import__("os").system("cls")
    else:
        _ = __import__("os").system("clear")

def prompt_enter():
    input("\nPress Enter to continue...")

def title(text):
    clear()
    print("=" * 60)
    print(text.center(60))
    print("=" * 60)

# -------------------------
# 1) ASCII Snake (curses)
# -------------------------
def snake_game():
    if curses is None:
        print("Snake requires the 'curses' module. On Windows run: pip install windows-curses")
        prompt_enter()
        return

    def run_snake(stdscr):
        curses.curs_set(0)
        sh, sw = stdscr.getmaxyx()
        # playable area (avoid tiny terminals)
        H, W = max(10, min(20, sh - 4)), max(20, min(60, sw - 4))
        box_top = (sh - H) // 2
        box_left = (sw - W) // 2
        win = curses.newwin(H, W, box_top, box_left)
        win.keypad(1)
        win.timeout(120)

        snk_x = W//4
        snk_y = H//2
        snake = [
            [snk_y, snk_x],
            [snk_y, snk_x-1],
            [snk_y, snk_x-2]
        ]
        key = curses.KEY_RIGHT

        food = [random.randint(1, H-2), random.randint(1, W-2)]
        win.addch(food[0], food[1], "@")

        score = 0

        while True:
            win.border()
            win.addstr(0, 2, f' Score: {score} ')
            next_key = win.getch()
            if next_key in [curses.KEY_RIGHT, curses.KEY_LEFT, curses.KEY_UP, curses.KEY_DOWN]:
                # avoid reverse gear
                if (key == curses.KEY_RIGHT and next_key != curses.KEY_LEFT) or \
                   (key == curses.KEY_LEFT and next_key != curses.KEY_RIGHT) or \
                   (key == curses.KEY_UP and next_key != curses.KEY_DOWN) or \
                   (key == curses.KEY_DOWN and next_key != curses.KEY_UP):
                    key = next_key
            elif next_key == 27:  # ESC to quit
                break

            head = snake[0].copy()
            if key == curses.KEY_RIGHT:
                head[1] += 1
            elif key == curses.KEY_LEFT:
                head[1] -= 1
            elif key == curses.KEY_UP:
                head[0] -= 1
            elif key == curses.KEY_DOWN:
                head[0] += 1

            # Collision with borders
            if head[0] == 0 or head[0] == H-1 or head[1] == 0 or head[1] == W-1:
                win.addstr(H//2, W//2 - 6, " GAME OVER ")
                win.getch()
                break

            # Collision with self
            if head in snake:
                win.addstr(H//2, W//2 - 6, " GAME OVER ")
                win.getch()
                break

            snake.insert(0, head)

            if head == food:
                score += 1
                while True:
                    new_food = [random.randint(1, H-2), random.randint(1, W-2)]
                    if new_food not in snake:
                        food = new_food
                        break
                win.addch(food[0], food[1], "@")
            else:
                tail = snake.pop()
                win.addch(tail[0], tail[1], ' ')

            win.addch(head[0], head[1], '#')

    title("🐍 ASCII SNAKE")
    print("Controls: Arrow keys to move. Press ESC to quit.")
    time.sleep(1)
    print("Starting in 2 seconds... Get your terminal focused!\n")
    time.sleep(2)
    curses.wrapper(run_snake)
    prompt_enter()

# -------------------------
# 2) Shooting Game (text)
# -------------------------
def shooting_game():
    title("🎯 SHOOTING RANGE")
    width = 30
    rows = 6
    shots = 6
    score = 0

    print("A target moves across columns. Choose column number to shoot (1..{})".format(width))
    print("You have {} shots. Hit the target to score.".format(shots))
    prompt_enter()

    target_row = random.randint(1, rows)
    # target will move left-right on a random row
    direction = 1
    pos = random.randint(1, width)
    for s in range(1, shots + 1):
        clear()
        print(f"Shot {s}/{shots} | Score: {score}\n")
        # display rows
        for r in range(1, rows+1):
            line = ""
            for c in range(1, width+1):
                if r == target_row and c == pos:
                    line += "T"
                else:
                    line += "."
            print(line)
        try:
            col = int(input("\nEnter column to shoot (1-{}): ".format(width)))
        except Exception:
            print("Invalid input! Counted as miss.")
            col = -1

        if col == pos:
            print("🎯 HIT! +10 points")
            score += 10
            # respawn target elsewhere
            target_row = random.randint(1, rows)
            pos = random.randint(1, width)
            direction = random.choice([-1, 1])
        else:
            print("💥 Miss!")
        time.sleep(1)
        # move target
        pos += direction
        if pos <= 1 or pos >= width:
            direction *= -1
            pos += direction
    print(f"\nGame over! Final score: {score}")
    prompt_enter()

# -------------------------
# 3) Treasure Hunt (text adventure)
# -------------------------
def treasure_hunt():
    title("✨ TREASURE HUNT")
    print("You're on a small island. Find the hidden treasure by choosing directions.")
    treasure_room = random.choice(["cave", "tree", "shore"])
    inventory = []
    rooms_visited = set()

    def describe(location):
        if location == "beach":
            return "A sunny beach. There's a path into the trees to the north."
        if location == "forest":
            return "Thick trees. You can go east to a clearing or south back to beach."
        if location == "clearing":
            return "A clearing with a strange stone. There's a hollow under it."
        if location == "cave":
            return "A dark cave. It smells like treasure... or bats."
        if location == "tree":
            return "A tall tree with a rope swing. You notice something shiny in the branches."
        if location == "shore":
            return "A rocky shore. Tides reveal a chest occasionally."
        return "An empty spot."

    location = "beach"
    turns = 0
    while True:
        clear()
        turns += 1
        print(f"Location: {location.upper()}")
        print(describe(location))
        print("\nInventory:", inventory)
        if location == treasure_room:
            # chance to find treasure
            found = random.random() < 0.7 or "map" in inventory
            if found:
                print("\n🎉 You found the TREASURE! Congratulations!")
                print(f"Took you {turns} turns.")
                prompt_enter()
                return
        print("\nActions: move [north/south/east/west], search, take [item], quit")
        cmd = input("> ").strip().lower()
        if cmd.startswith("move"):
            dir = cmd.split(" ")[1] if len(cmd.split()) > 1 else ""
            if location == "beach" and dir == "north":
                location = "forest"
            elif location == "forest" and dir == "east":
                location = "clearing"
            elif location == "forest" and dir == "south":
                location = "beach"
            elif location == "clearing" and dir == "west":
                location = "forest"
            elif location == "beach" and dir == "east":
                location = "shore"
            elif location == "shore" and dir == "west":
                location = "beach"
            elif location == "clearing" and dir == "north":
                location = "cave"
            elif location == "beach" and dir == "west":
                location = "tree"
            else:
                print("You can't go that way.")
                time.sleep(1)
        elif cmd == "search":
            # random finds
            if location == "clearing" and "map" not in inventory and random.random() < 0.6:
                print("You found a torn MAP piece!")
                inventory.append("map")
            elif location == "tree" and "rope" not in inventory and random.random() < 0.5:
                print("You found a STRONG rope in the branches.")
                inventory.append("rope")
            elif location == "shore" and random.random() < 0.4:
                print("You found a shiny COIN.")
                inventory.append("coin")
            else:
                print("You found nothing useful.")
            time.sleep(1.4)
        elif cmd.startswith("take"):
            print("There is nothing special to 'take' here right now.")
            time.sleep(1)
        elif cmd == "quit":
            print("You gave up the hunt. Maybe next time!")
            prompt_enter()
            return
        else:
            print("I don't understand that action.")
            time.sleep(1)

# -------------------------
# 4) IQ / Memory Challenge
# -------------------------
def memory_game():
    title("🧠 MEMORY CHALLENGE")
    print("You'll see a sequence of symbols for a short time. Repeat them exactly.")
    levels = 1
    max_level = 6
    symbols = "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
    while levels <= max_level:
        seq = "".join(random.choice(symbols) for _ in range(levels + 2))
        clear()
        print(f"Level {levels} — memorize the sequence:")
        print("\n" + " ".join(seq))
        time.sleep(max(1.5, len(seq) * 0.6))
        clear()
        ans = input("Type the sequence exactly (no spaces): ").strip().upper()
        if ans == seq:
            print("✅ Correct! Moving to next level.")
            levels += 1
            time.sleep(1)
        else:
            print(f"❌ Wrong! The correct sequence was: {seq}")
            print(f"You reached level {levels}.")
            prompt_enter()
            return
    print("🏆 Amazing! You beat all levels!")
    prompt_enter()

# -------------------------
# 5) Roast Battle
# -------------------------
def roast_battle():
    title("🤣 ROAST BATTLE")
    roasts = [
        "I'd explain it to you, but I left my crayons at home.",
        "You're proof that even Google doesn't have all the answers.",
        "I'd call you a tool, but even they serve a purpose.",
        "If brains were dynamite you wouldn't have enough to blow your nose.",
        "You're like a cloud. When you disappear, it's a beautiful day.",
        "Your secrets are always safe with me. I never even listen.",
        "I've seen salads dressed better than you.",
        "You're the reason the gene pool needs a lifeguard.",
        "Some drink from the fountain of knowledge; you only gargled.",
        "You're a gray sprinkle on a rainbow cupcake."
    ]
    print("Welcome to Roast Battle! Two players exchange roasts.")
    p1 = input("Player 1 name: ").strip() or "Player1"
    p2 = input("Player 2 name: ").strip() or "Player2"
    rounds = 3
    scores = {p1: 0, p2: 0}
    for r in range(1, rounds+1):
        clear()
        print(f"Round {r} — {p1}'s turn to roast {p2}")
        input("Press Enter to generate a roast...")
        roast = random.choice(roasts)
        print(f"\n{p1} -> {p2}: {roast}")
        vote = input("\nWho laughed more? (1 for P1's roast, 2 for P2's roast, 0 nobody): ").strip()
        if vote == "1":
            scores[p1] += 1
        elif vote == "2":
            scores[p2] += 1
        time.sleep(1)
        # now P2's roast
        clear()
        print(f"Round {r} — {p2}'s turn to roast {p1}")
        input("Press Enter to generate a roast...")
        roast = random.choice(roasts)
        print(f"\n{p2} -> {p1}: {roast}")
        vote = input("\nWho laughed more? (1 for P2's roast, 2 for P1's roast, 0 nobody): ").strip()
        if vote == "1":
            scores[p2] += 1
        elif vote == "2":
            scores[p1] += 1
        time.sleep(1)

    clear()
    print("Final Scores:")
    print(f"{p1}: {scores[p1]}\n{p2}: {scores[p2]}")
    if scores[p1] > scores[p2]:
        print(f"🏆 {p1} wins the Roast Battle!")
    elif scores[p2] > scores[p1]:
        print(f"🏆 {p2} wins the Roast Battle!")
    else:
        print("🤝 It's a tie — both are equally savage!")
    prompt_enter()

# -------------------------
# 6) Multiplayer Text Battle (local 2-player)
# -------------------------
def multiplayer_battle():
    title("⚔️ MULTIPLAYER TEXT BATTLE (Local)")
    p1 = input("Player 1 name: ").strip() or "Player1"
    p2 = input("Player 2 name: ").strip() or "Player2"
    hp = {p1: 100, p2: 100}
    turn = p1 if random.random() < 0.5 else p2
    print(f"{turn} goes first!")
    prompt_enter()

    while hp[p1] > 0 and hp[p2] > 0:
        clear()
        print(f"{p1}: {hp[p1]} HP | {p2}: {hp[p2]} HP")
        print(f"\n{turn}'s turn.")
        print("Actions: attack, heavy (more dmg but 30% miss), heal")
        act = input("> ").strip().lower()
        opponent = p2 if turn == p1 else p1
        if act == "attack":
            dmg = random.randint(8, 18)
            hp[opponent] -= dmg
            print(f"{turn} hits {opponent} for {dmg} dmg!")
        elif act == "heavy":
            if random.random() < 0.3:
                print("Missed heavy attack!")
            else:
                dmg = random.randint(20, 32)
                hp[opponent] -= dmg
                print(f"Heavy hit! {opponent} loses {dmg} HP!")
        elif act == "heal":
            heal = random.randint(10, 20)
            hp[turn] += heal
            if hp[turn] > 120: hp[turn] = 120
            print(f"{turn} heals for {heal} HP!")
        else:
            print("Invalid action. You lose your turn.")

        time.sleep(1.2)
        if hp[opponent] <= 0:
            print(f"\n{opponent} has fallen. {turn} wins! 🏆")
            break
        # switch
        turn = opponent
    prompt_enter()

# -------------------------
# 7) Horror Escape (text)
# -------------------------
def horror_escape():
    title("👻 HORROR ESCAPE")
    print("You wake up in a creepy abandoned mansion. Escape before midnight.")
    inventory = []
    rooms = {
        "foyer": {"desc":"A dim foyer. Doors lead north and east.", "north":"hall", "east":"kitchen"},
        "hall": {"desc":"A long hall with portraits. There's a locked door to the north.", "south":"foyer", "north":"attic"},
        "kitchen": {"desc":"Old kitchen. Smells. There's a drawer.", "west":"foyer"},
        "attic": {"desc":"Dusty attic. A lantern hangs here.", "south":"hall"}
    }
    location = "foyer"
    time_left = 10  # turns until midnight (bad ending)

    while time_left > 0:
        clear()
        print(f"Location: {location} | Time left: {time_left}")
        print(rooms[location]["desc"])
        print("Inventory:", inventory)
        action = input("\nActions: move [dir], search, use [item], quit\n> ").strip().lower()
        time_left -= 1

        if action.startswith("move"):
            parts = action.split()
            if len(parts) < 2:
                print("Move where?")
                time.sleep(1)
                continue
            d = parts[1]
            if d in rooms[location]:
                location = rooms[location][d]
                # random scary event
                if random.random() < 0.15:
                    print("\nA distant whisper chills your spine... but nothing else happens.")
                    time.sleep(1.2)
            else:
                print("You can't go that way.")
                time.sleep(1)
        elif action == "search":
            if location == "kitchen" and "key" not in inventory:
                print("You find a rusty key in a drawer.")
                inventory.append("key")
            elif location == "attic" and "lantern" not in inventory:
                print("You take the lantern. It might light dark places.")
                inventory.append("lantern")
            elif location == "hall" and "key" in inventory and random.random() < 0.5:
                print("You notice the locked door is now slightly ajar.")
            else:
                print("You search but find nothing useful.")
            time.sleep(1.2)
        elif action.startswith("use"):
            parts = action.split()
            if len(parts) < 2:
                print("Use what?")
                time.sleep(1)
                continue
            item = parts[1]
            if item == "key" and "key" in inventory and location == "hall":
                if random.random() < 0.8:
                    print("The key turns. The locked door opens to a staircase down... You escape! 🎉")
                    prompt_enter()
                    return
                else:
                    print("The key broke in the lock!")
                    inventory.remove("key")
            elif item == "lantern" and "lantern" in inventory:
                print("You light the lantern. The rooms feel less menacing.")
                # small chance to reveal exit
                if random.random() < 0.3:
                    print("In the light you notice a loose tile in the foyer that hides a trapdoor exit.")
                    if "trapdoor key" not in inventory:
                        inventory.append("trapdoor key")
                time.sleep(1.5)
            else:
                print("You can't use that here.")
                time.sleep(1)
        elif action == "quit":
            print("You gave up. The mansion keeps your secrets.")
            prompt_enter()
            return
        else:
            print("Unknown action.")
            time.sleep(1)
    # time ran out
    clear()
    print("The clock strikes midnight... Something took you in the dark. 😱")
    prompt_enter()

# -------------------------
# Main menu
# -------------------------
def main_menu():
    while True:
        clear()
        print("="*60)
        print(" PARTY GAMES — Send this file to your friends and play! ".center(60))
        print("="*60)
        print("""
Choose a game:
1) Snake (ASCII) 🐍
2) Shooting Range 🎯
3) Treasure Hunt ✨
4) Memory / IQ Challenge 🧠
5) Roast Battle 🤣
6) Multiplayer Text Battle ⚔️
7) Horror Escape 👻
8) Rock-Paper-Scissors (Continuous) ✂️
0) Exit
        """)
        choice = input("Enter choice: ").strip()
        if choice == "1":
            snake_game()
        elif choice == "2":
            shooting_game()
        elif choice == "3":
            treasure_hunt()
        elif choice == "4":
            memory_game()
        elif choice == "5":
            roast_battle()
        elif choice == "6":
            multiplayer_battle()
        elif choice == "7":
            horror_escape()
        elif choice == "8":
            # simple continuous RPS
            title("✂️ ROCK PAPER SCISSORS (Continuous)")
            print("Type rock, paper, scissors to play. Type exit to stop.")
            choices = ["rock", "paper", "scissors"]
            wins = losses = ties = 0
            while True:
                user = input("Your choice: ").lower()
                if user == "exit":
                    break
                if user not in choices:
                    print("Invalid.")
                    continue
                comp = random.choice(choices)
                print(f"Computer: {comp}")
                if user == comp:
                    print("Tie!")
                    ties += 1
                elif (user == "rock" and comp == "scissors") or \
                     (user == "paper" and comp == "rock") or \
                     (user == "scissors" and comp == "paper"):
                    print("You win!")
                    wins += 1
                else:
                    print("You lose!")
                    losses += 1
                print(f"W:{wins} L:{losses} T:{ties}")
            prompt_enter()
        elif choice == "0":
            print("Goodbye — have fun with your friends! 🎉")
            time.sleep(0.6)
            break
        else:
            print("Invalid selection.")
            time.sleep(1)

if __name__ == "__main__":
    main_menu()


   PARTY GAMES — Send this file to your friends and play!   

Choose a game:
1) Snake (ASCII) 🐍
2) Shooting Range 🎯
3) Treasure Hunt ✨
4) Memory / IQ Challenge 🧠
5) Roast Battle 🤣
6) Multiplayer Text Battle ⚔️
7) Horror Escape 👻
8) Rock-Paper-Scissors (Continuous) ✂️
0) Exit
        
                      🎯 SHOOTING RANGE                      
A target moves across columns. Choose column number to shoot (1..30)
You have 6 shots. Hit the target to score.
Shot 1/6 | Score: 0

..............................
..............................
..............................
..............................
......T.......................
..............................
🎯 HIT! +10 points
Shot 2/6 | Score: 10

..............................
..............................
...T..........................
..............................
..............................
..............................
🎯 HIT! +10 points
Shot 3/6 | Score: 20

..............................
..............................
.......